In [383]:
#use the data to know the daily nutrition of different age ang gender
!pip install eep153_tools >/dev/null
!pip install gspread_pandas >/dev/null
!pip install -r requirements.txt >/dev/null
# Import libraries
import numpy as np
import pandas as pd
import warnings
import fooddatacentral as fdc
from eep153_tools.sheets import read_sheets
from  scipy.optimize import linprog

In [384]:
# Read diet minimum data
diet_min = pd.read_csv('cleanedmins.csv', index_col = 2)

# Read diet maximum data
diet_max = pd.read_excel("Dietary Requirements.xlsx",sheet_name='diet_maximums',index_col=0)
diet_max

#only have sodium and enery

,Source,C 1-3,F 4-8,M 4-8,F 9-13,M 9-13,F 14-18,M 14-18,F 19-30,M 19-30,F 31-50,M 31-50,F 51+,M 51+
Nutrition,,,,,,,,,,,,,,
"Sodium, Na",UL,1500,1900,1900,2200,2200,2300,2300,2300,2300,2300,2300,2300,2300
Energy,NaN,2500,2500,2500,2800,3000,3100,3100,3100,3100,3100,3100,3100,3100


In [385]:
# define two function use age and gender
def diet_minimum(age, sex):
    
    if age <= 3:
        col = 'C 1-3'
    age_ranges = [(4,8),(9,13),(14,18),(19,30),(31,50),(50,100)]
    for age_range in age_ranges:
        if age >= age_range[0] and age <= age_range[1]:
            col = sex + ' ' + str(age_range[0]) + '-' + str(age_range[1])
    return pd.Series(diet_min[col])  

def diet_maximum(age, sex):
    
    if age <= 3:
        col = 'C 1-3'
    age_ranges = [(4,8),(9,13),(14,18),(19,30),(31,50),(50,100)]
    for age_range in age_ranges:
        if age >= age_range[0] and age <= age_range[1]:
            col = sex + ' ' + str(age_range[0]) + '-' + str(age_range[1])
    return pd.Series(diet_min[col])  

In [386]:
diet_minimum(age=24,sex='F')

Nutrition
Energy                         2000.0
Protein                          46.0
Fiber, total dietary             28.0
Calcium, Ca                    1000.0
Carbohydrate, by difference     130.0
Iron, Fe                         18.0
Potassium, K                   4700.0
Name: F 19-30, dtype: float64

we use trader_joes data

Now, the prices we observe can be for lots of different quantities and units. The FDC database basically wants everything in either hundreds of grams (hectograms) or hundreds of milliliters (deciliters). We use the units function to convert all foods to either deciliters or hectograms, to match FDC database:

In [387]:
trader_joes = pd.read_csv("trader_joes_product_details_cleaned2.csv", index_col = 0)
trader_joes

,price,package_size,serving_size,calories_per_serving,calculated_calories,Total Fat,Saturated Fat,Trans Fat,Cholesterol,Sodium,Total Carbohydrate,Dietary Fiber,Total Sugars,Added Sugars,Protein,Vitamin D,Calcium,Iron,Potassium
name,,,,,,,,,,,,,,,,,,,
Chicken Chile Verde Burritos,$3.99,396.90 g,198.00 g,810.0,825.0,29.00 g,8.00 g,0.00 g,0.07 g,1.95 g,102.00 g,4.00 g,2.00 g,0.00 g,39.00 g,0.000000 g,0.27 g,0.01 g,0.44 g
Nocciolini Tiny Hazelnut Meringues,$2.69,99.23 g,28.00 g,150.0,148.0,8.00 g,0.50 g,0.00 g,0.00 g,0.01 g,16.00 g,1.00 g,16.00 g,15.00 g,3.00 g,0.000000 g,0.02 g,0.00 g,0.11 g
Crispy Oatmeal Cookies with Sea Salt,$3.99,198.45 g,30.00 g,120.0,121.0,5.00 g,3.00 g,0.00 g,0.02 g,0.13 g,17.00 g,0.00 g,8.00 g,8.00 g,2.00 g,0.000000 g,0.01 g,0.00 g,0.02 g
Chocolate Coffee Flavored Granola,$3.99,340.20 g,58.00 g,110.0,260.0,8.00 g,1.50 g,0.00 g,0.00 g,0.12 g,42.00 g,5.00 g,15.00 g,15.00 g,5.00 g,0.000000 g,0.02 g,0.00 g,0.24 g
Avocado Mash,$3.49,226.80 g,30.00 g,45.0,56.0,4.00 g,0.50 g,0.00 g,0.00 g,0.09 g,3.00 g,2.00 g,0.00 g,0.00 g,2.00 g,0.000000 g,0.00 g,0.00 g,0.14 g
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Gluten Free Ginger Turmeric Granola,$3.49,340.20 g,60.00 g,270.0,273.0,9.00 g,6.00 g,0.00 g,0.00 g,0.14 g,43.00 g,5.00 g,14.00 g,13.00 g,5.00 g,0.000000 g,0.03 g,0.00 g,0.18 g
Chile & Garlic Cashews,$3.99,226.80 g,30.00 g,170.0,177.0,13.00 g,2.50 g,0.00 g,0.00 g,0.15 g,10.00 g,1.00 g,1.00 g,0.00 g,5.00 g,0.000000 g,0.01 g,0.00 g,0.17 g
Gone Berry Crazy!,$2.79,150.25 g,38.00 g,100.0,99.5,3.50 g,2.50 g,0.00 g,0.00 g,0.02 g,15.00 g,2.00 g,12.00 g,5.00 g,2.00 g,0.000000 g,0.03 g,0.00 g,0.09 g


In [388]:
print("Diet Min Nutrients:", diet_min.index.tolist())
print("Trader Joe's Columns:", trader_joes.columns.tolist())

Diet Min Nutrients: ['Energy', 'Protein', 'Fiber, total dietary', 'Calcium, Ca', 'Carbohydrate, by difference', 'Iron, Fe', 'Potassium, K']
Trader Joe's Columns: ['price', 'package_size', 'serving_size', 'calories_per_serving', 'calculated_calories', 'Total Fat', 'Saturated Fat', 'Trans Fat', 'Cholesterol', 'Sodium', 'Total Carbohydrate', 'Dietary Fiber', 'Total Sugars', 'Added Sugars', 'Protein', 'Vitamin D', 'Calcium', 'Iron', 'Potassium']


In [389]:
# Uniformly removes "g", "mg", "%", "$" from all columns
trader_joes = trader_joes.astype(str).apply(lambda x: x.str.replace(r"\s*g", "", regex=True)
                                                          .str.replace(r"\s*mg", "", regex=True)
                                                          .str.replace(r"\s*%", "", regex=True)
                                                          .str.replace(r"\$", "", regex=True))
# Converts all numeric columns back to float
trader_joes = trader_joes.apply(pd.to_numeric, errors='coerce')

# Calculate 'calculated_calories'
trader_joes["calculated_calories"] = (trader_joes["package_size"] / trader_joes["serving_size"]) * trader_joes["calories_per_serving"]

trader_joes

,price,package_size,serving_size,calories_per_serving,calculated_calories,Total Fat,Saturated Fat,Trans Fat,Cholesterol,Sodium,Total Carbohydrate,Dietary Fiber,Total Sugars,Added Sugars,Protein,Vitamin D,Calcium,Iron,Potassium
name,,,,,,,,,,,,,,,,,,,
Chicken Chile Verde Burritos,3.99,396.90,198.0,810.0,1623.681818,29.0,8.0,0.0,0.07,1.95,102.0,4.0,2.0,0.0,39.0,0.0,0.27,0.01,0.44
Nocciolini Tiny Hazelnut Meringues,2.69,99.23,28.0,150.0,531.589286,8.0,0.5,0.0,0.00,0.01,16.0,1.0,16.0,15.0,3.0,0.0,0.02,0.00,0.11
Crispy Oatmeal Cookies with Sea Salt,3.99,198.45,30.0,120.0,793.800000,5.0,3.0,0.0,0.02,0.13,17.0,0.0,8.0,8.0,2.0,0.0,0.01,0.00,0.02
Chocolate Coffee Flavored Granola,3.99,340.20,58.0,110.0,645.206897,8.0,1.5,0.0,0.00,0.12,42.0,5.0,15.0,15.0,5.0,0.0,0.02,0.00,0.24
Avocado Mash,3.49,226.80,30.0,45.0,340.200000,4.0,0.5,0.0,0.00,0.09,3.0,2.0,0.0,0.0,2.0,0.0,0.00,0.00,0.14
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Gluten Free Ginger Turmeric Granola,3.49,340.20,60.0,270.0,1530.900000,9.0,6.0,0.0,0.00,0.14,43.0,5.0,14.0,13.0,5.0,0.0,0.03,0.00,0.18
Chile & Garlic Cashews,3.99,226.80,30.0,170.0,1285.200000,13.0,2.5,0.0,0.00,0.15,10.0,1.0,1.0,0.0,5.0,0.0,0.01,0.00,0.17
Gone Berry Crazy!,2.79,150.25,38.0,100.0,395.394737,3.5,2.5,0.0,0.00,0.02,15.0,2.0,12.0,5.0,2.0,0.0,0.03,0.00,0.09


In [390]:
nutrient_mapping = {
    "calculated_calories": "Energy",
    "Protein": "Protein",
    "Dietary Fiber": "Fiber, total dietary",
    "Calcium": "Calcium, Ca",
    "Total Carbohydrate": "Carbohydrate, by difference",
    "Iron": "Iron, Fe",
    "Potassium": "Potassium, K"
}

# rename Trader Joe's dataset
trader_joes.rename(columns=nutrient_mapping, inplace=True)

# check diet_min and trader_joes index
common_nutrients = list(set(diet_min.index) & set(trader_joes.columns))
print("Matching Nutrients:", common_nutrients)

Matching Nutrients: ['Energy', 'Iron, Fe', 'Potassium, K', 'Calcium, Ca', 'Carbohydrate, by difference', 'Fiber, total dietary', 'Protein']


In [391]:
import numpy as np
import pandas as pd

def compute_food_cost(age, sex, diet_min, trader_joes):
    """
        Calculate the number of servings of each food item needed to meet the minimum Energy requirements and calculate the total cost.

        Parameters:
            - age: int indicates the age of the user
            - sex: str, 'M' or 'F'
            - diet_min: DataFrame, which contains the minimum nutrient requirement
            - trader_joes: DataFrame, which contains Trader Joe's food data

        Back:
            - A DataFrame containing:
            - Food name
            - Number of copies required
            - Price per serving
            - Total cost
    """
    energy_required = float(diet_minimum(age, sex).loc["Energy"])
    food_energy = trader_joes["Energy"].astype(float)
    food_servings_needed = np.ceil(energy_required / food_energy)
    food_prices = trader_joes["price"].astype(float)
    total_cost = food_servings_needed * food_prices
    result_df = pd.DataFrame({
        "Food": trader_joes.index,
        "Servings Needed": food_servings_needed,
        "Price per Serving": food_prices,
        "Total Cost": total_cost
    })
    result_df = result_df.sort_values(by="Total Cost")
    return result_df
food_cost_df = compute_food_cost(age=24, sex='F', diet_min=diet_min, trader_joes=trader_joes)
food_cost_df

cheapest_food = food_cost_df.iloc[0]
print("Cheapest Food Option:\n", cheapest_food)

Cheapest Food Option:
 Food                 Chocolatey Dipping Kit
Servings Needed                         2.0
Price per Serving                      0.01
Total Cost                             0.02
Name: Chocolatey Dipping Kit, dtype: object


In [392]:
from scipy.optimize import linprog
import warnings
import numpy as np
import pandas as pd

def solve_subsistence_problem(FoodNutrients, Prices, dietmin, max_weight=None, tol=1e-6):
    """
    Solve the minimum cost diet problem using linear programming (without diet_max constraints).
    
    Parameters:
    - FoodNutrients: DataFrame, each row is a food item, each column is a nutrient
    - Prices: Series, price of each food item
    - dietmin: Series, minimum required nutrients
    - max_weight: float, optional, max weight allowed for the diet
    - tol: float, numerical tolerance for small values

    Returns:
    - result: Optimization result object
    - diet: Series, optimal food combination
    """

    #Make sure there is no NaN
    Prices = pd.Series(Prices, index=FoodNutrients.index).dropna()
    FoodNutrients = FoodNutrients.loc[Prices.index].fillna(0).astype(float)  # NaN -> 0
    dietmin = dietmin.fillna(0).astype(float)  # NaN -> 0

    
    A = FoodNutrients.loc[FoodNutrients.index.intersection(dietmin.index)]
    A = A.reindex(dietmin.index, axis=0)
    A.replace([np.inf, -np.inf], 0, inplace=True)
    dietmin.replace([np.inf, -np.inf], 0, inplace=True)
    A = A.reindex(Prices.index, axis=1)
    A = A.reindex(dietmin.index, axis=0)
    # Minimum requirements
    b = dietmin

    if max_weight is not None:
        A.loc['Hectograms'] = -1
        b.loc['Hectograms'] = -max_weight

    # Operational linear programming
    result = linprog(Prices, A_ub=-A, b_ub=-b, method='highs')

    if result.success:
        diet = pd.Series(result.x, index=Prices.index)
    else:
        warnings.warn(result.message)
        diet = pd.Series(result.x, index=Prices.index) * np.nan

    return result, diet

In [393]:
Prices = trader_joes["price"].astype(float)
FoodNutrients = trader_joes[["Energy", "Protein", "Fiber, total dietary", "Calcium, Ca", 
                             "Carbohydrate, by difference", "Iron, Fe", "Potassium, K"]].fillna(0).astype(float)


sex, age = 'F', 24
diet_min_values = diet_min[diet_min.columns[diet_min.columns.str.contains(f"{sex}")]].iloc[:, 0].fillna(0).astype(float)

# operation optimization
optimal_result, optimal_diet = solve_subsistence_problem(FoodNutrients, Prices, diet_min_values)


print(f"\nCost of diet for {sex} {age} is ${optimal_result.fun:.2f} per day.\n")
print("\nDiet (in 100s of grams or milliliters):")
selected_foods = optimal_diet[optimal_diet > 1e-6]
print(selected_foods)
tab = pd.DataFrame({"Outcome": np.abs(FoodNutrients).dot(optimal_diet), "Recommendation": np.abs(diet_min_values)})
print("\nWith the following nutritional outcomes of interest:")
print(tab)


ValueError: Invalid input for linprog: A_ub must not contain values inf, nan, or None